# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient. We will also store the timestamp of each email. 

In [1]:
import networkx as nx

#!head assets/email_network.txt

In [4]:
!head -5 assets/email_network.txt

#Sender	Recipient	time
1	2	1262454010
1	3	1262454010
1	4	1262454010
1	5	1262454010


### Question 1

Using networkx, load up the directed multigraph from `assets/email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [2]:
import pandas as pd
def answer_one():
    
    df = pd.read_csv('assets/email_network.txt', delimiter='\t')
    df[['#Sender', 'Recipient']] = df[['#Sender', 'Recipient']].astype(str)
    G = nx.from_pandas_edgelist(df, '#Sender', 'Recipient', create_using = nx.MultiDiGraph)
    return G
answer_one()

In [ ]:
ans_one = answer_one()

### Question 2

How many employees are represented in the network?

How many `sender`->`recipient` pairs of employees are there in the network such that `sender` sent at least one email to `recipient`? Note that even if a `sender` sent multiple messages to a `recipient`, they should only be counted once. You should **not** exclude cases where an employee sent emails to themselves from this [email] count. 

*This function should return a tuple with two integers (#employees, # `sender`->`recipient` pairs).*

In [3]:
df = pd.read_csv('assets/email_network.txt', delimiter='\t')
df[['#Sender', 'Recipient']] = df[['#Sender', 'Recipient']].astype(str)

df.head()

,#Sender,Recipient,time
0,1,2,1262454010
1,1,3,1262454010
2,1,4,1262454010
3,1,5,1262454010
4,1,6,1262454010


5784

In [86]:
import numpy as np
def answer_two():
    
    n_employees =   len(answer_one().nodes()) # len(np.unique(df[['#Sender', 'Recipient']]))
                                                # or len(np.unique(list(np.unique(df['Recipient'])) + list(np.unique(df['#Sender']))))
    
    n_unique_pairs = len(set(list(answer_one().edges()))) #df['Sorted'] = df.apply(lambda row: sorted([row[0], row[1]]), axis=1)
                                                #len(np.unique(df['Sorted']))
    
    return (n_employees, n_unique_pairs)

answer_two()

(167, 5784)

In [ ]:
ans_two = answer_two()


### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the reciever, but not viceversa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [5]:
# PART 1
t = pd.DataFrame(answer_one().nodes())
t['is_sender'] = t[0].isin(df['#Sender'])
t = t.rename(columns={0: 'node_n'})
not_sender_nodes = t[t['is_sender']==False]
not_sender_nodes

,node_n,is_sender
101,102,False
113,114,False
115,116,False
124,125,False
125,126,False
126,127,False
129,130,False
130,131,False
131,132,False
132,133,False


In [6]:
def answer_three():
    # PART 1: There are False values, meaning that there are nodes that are not senders but just recievers.
    t = pd.DataFrame(answer_one().nodes())
    t['is_sender'] = t[0].isin(df['#Sender'])
    t = t.rename(columns={0: 'node_n'})
    not_sender_nodes = t[t['is_sender']==False]
    
    # PART 2: 
    # Yes
    
    return ('No', 'Yes')

answer_three()

('False', 'True')

In [ ]:
ans_three = answer_three()


### Question 4

How many nodes are in the largest weakly connected component of the graph?

*This function should return an int.*

In [37]:
def answer_four():
    
    return len(max(nx.weakly_connected_components(answer_one()), key=len))

answer_four()

167

In [ ]:
ans_four = answer_four()

### Question 5

How many nodes are in the largest strongly connected component?

*This function should return an int*

In [36]:
def answer_five():
    
    return len(max(nx.strongly_connected_components(answer_one()), key=len))

answer_five()

126

In [ ]:
ans_five = answer_five()


### Question 6

Using the NetworkX functions `strongly_connected_components` and `subgraph`, find the subgraph of nodes in the largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [4]:
def answer_six():
    
    scc = max(nx.strongly_connected_components(answer_one()), key=len)
    
    G_sc = answer_one().subgraph(scc)
              
    return G_sc

answer_six()

In [ ]:
ans_six = answer_six()
assert type(ans_six) == nx.MultiDiGraph , "Your return type should be a MultiDiGraph object"


### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [77]:
def answer_seven():
    
    G_sc = answer_six()
    return nx.average_shortest_path_length(G_sc)

answer_seven()

1.6461587301587302

In [ ]:
ans_seven = answer_seven()


### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [43]:
def answer_eight():
    
    G_sc = answer_six()
    
    return nx.diameter(G_sc)

answer_eight()

3

In [44]:
# OR THIS OTHER WAY, WHICH IS A BIT LONGER

def answer_t():
    
    G_sc = answer_six()
    eccentricity = nx.eccentricity(G_sc)
    
    max_ecc = max(eccentricity, key=eccentricity.get)
    
    return eccentricity[max_ecc]

answer_t()

3

In [ ]:
ans_eight = answer_eight()


### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [65]:
def answer_nine():
    
    G_sc = answer_six()
    eccentricity = nx.eccentricity(G_sc)
    
    diam = nx.diameter(G_sc)
    
    diam_nodes = []
    
    for node in eccentricity.items():
        if node[1] == diam:
            diam_nodes.append(node[0])
    
    return set(diam_nodes) # OR JUST WRITE set(nx.periphery(G_sc))

answer_nine()

{'129', '134', '97'}

In [ ]:
ans_nine = answer_nine()
assert type(ans_nine) == set, "Student answer must return a set"

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [70]:
def answer_ten():
    
    G_sc = answer_six()
    
    return set(nx.center(G_sc))

answer_ten()

{'38'}

In [ ]:
ans_ten = answer_ten()
assert type(ans_ten) == set, "Student answer must return a set"

### Question 11

Which node in G_sc has the most shortest paths to other nodes whose distance equal the diameter of G_sc?


For the node with the most such shortest paths, how many of these paths are there?


*This function should return a tuple (name of node, number of paths).*

In [20]:
def answer_eleven():
    
    G_sc = answer_six()
    peri = nx.periphery(G_sc)
    diam = nx.diameter(G_sc)
    dct = {}

    for node in peri:
        spl = nx.shortest_path_length(G_sc, source=node)
        path_length_diam = [i for i in spl.values() if i == diam]
        dct[node] = len(path_length_diam)

    max_value = max(dct.items(), key=lambda x: x[1])
    return max_value

answer_eleven()

('97', 63)

In [ ]:
ans_eleven = answer_eleven()
assert type(ans_eleven) == tuple, "Student answer must be a tuple"

### Question 12

Suppose you want to prevent communication flow from the node that you found in question 11 to node 10. What is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or 10)? 

*This function should return an integer.*

In [59]:
nx.minimum_node_cut(G_sc, s='10', t='97')

{'14', '32', '37', '38', '45', '46'}

In [27]:
shortest_path = nx.shortest_path(G_sc, source='97')['10']
nodes = [i for i in shortest_path if i != '97' and i != '10']
len(nodes)

1

In [48]:
def answer_twelve():
    
    G_sc = answer_six()
    
    answer = nx.node_connectivity(G_sc, s=answer_eleven()[0], t='10') #or len(nx.minimum_node_cut(G_sc, s='97', t='10'))

    return answer

answer_twelve()

6

In [ ]:
ans_twelve = answer_twelve()


### Question 13

Convert the graph G_sc into an undirected graph by removing the direction of the edges of G_sc. Call the new graph G_un. 


*This function should return a networkx Graph.*

In [68]:
def answer_thirteen():
    
    G_sc = answer_six()
    G_un = nx.Graph(G_sc)
    return G_un

nx.is_directed(answer_thirteen())

False

In [ ]:
ans_thirteen = answer_thirteen()
assert type(ans_thirteen) == nx.Graph , "Your return type should be a Graph object"


### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*     
*Note: DO NOT round up your answer.*

In [72]:
def answer_fourteen():
    
    G_sc = answer_six()
    G_un = answer_thirteen()
    
    return (nx.transitivity(G_un), nx.average_clustering(G_un))

answer_fourteen()

(0.570111160700385, 0.6975272437231418)

In [ ]:
ans_fourteen = answer_fourteen()
assert type(ans_fourteen) == tuple, "Student answer must be a tuple"
